In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex
import math

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Course Timetable')

Restricted license - for non-production use only - expires 2026-11-23


In [2]:

# make a dict of sessions --- each course splits into as many sessions are there are
# during the week for that course
# add other course attributes as needed like the enrollment strength etc.
sessions = {}
n_sessions = len(sessions)

## Make 2 sets of binary variables --- one for timeslot allocation to sessions and another for room allocation to courses
$$S_{beds} = \left\{\begin{array}{ll}1 & \mbox{if session $s$ begins on day $d$ at time $b$ and ends at time $e$}\\0 & \mbox{Otherwise}\end{array}\right.$$
$b,e$ are indexes into the 'times' array and $d$ is an index into the 'days' array.
$$C_{cr} = \left\{\begin{array}{ll}1 & \mbox{if course $c$ is assigned room $r$}\\0 & \mbox{Otherwise}\end{array}\right.$$

In [3]:
def session_length(s:int):
    # get session length (in 15 min chunks) for the course corresponding to the session 's'
    return 6 # used the default value here

session_spans = {s: begin_end_pairs(session_length(s)) for s in range(n_sessions)}

S = tt.addVars(
    [
        (b,e,d,s)
        for s in range(n_sessions)
        for d in range(data.n_days)
        for b, e in session_spans[s]
    ],
    vtype=GRB.BINARY,
    name="S",
)

# Each course is assigned a classroom through these indicator variables
C = tt.addVars(
    [
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name="C",
)

## Hard Constraints
##### Each session gets exactly one slot during the week: $\forall s: \sum_{b,e,d} S_{beds}=1$
##### Each Course gets exactly one room: $\forall c: \sum_r C_{cr}=1$
##### Course enrollment cannot exceed room capacity: $\forall c, r: C_{cr}. \texttt{enrollment}(c)\leq \texttt{capacity}(r)$
##### At most one session assigned to any room at any given time & day: $\forall d, t: \sum_{c,r,s,b,e}C_{cr}.S_{beds}\leq  1$
$s$ is a session corresponding to course 'c', (b,e) is a valid time interval for session 's' containing time 't'. Note that $C_{cr}.S_{beds}=1$ if and only if course 'c' is assigned room 'r' and a session 's' for the course is scheduled on day 'd' to start at 'b' and end at 'e'. Clearly there can be at most one such session containing 't'.

In [4]:
# ----- Each session gets exactly one slot during the week ------------------
tt.addConstrs((S.sum('*','*','*',s) == 1 for s in range(n_sessions)),
              name='one_slot_per_session'
             )

# ----- Each course gets exactly one classroom --------------------
tt.addConstrs((C.sum(c,'*') == 1 for c in range(data.n_courses)),
              name='one_room_per_course'
             )

# ----- Course enrollment does not exceed the classroom capacity --------------------
def capacity(room:int):
    # returns the capacity of the room given the room id (in the 'classrooms' dataframe)
    return 1

def enrollment(c:int):
    # returns the enrollment strength of the course, given the course id (in the 'courses' dataframe)
    return 1

# Capacity constraints
tt.addConstrs((C[c,r]*enrollment(c) <= capacity(r)
              for c in range(data.n_courses)
              for r in range(data.n_rooms)),
              name='course_room_fit'
             )

# -------------- At most one session assigned to any room at any given time ---------------
def classes_for_course(course:int):
    # Return the list of class indices for the given course
    return []

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in session_spans[s] if not overlap(b,e,t)]

tt.addConstrs((gp.quicksum(C[c,r]*S[b,e,d,s]
                          for c in range(data.n_courses)
                          for r in range(data.n_rooms)
                          for s in classes_for_course(c)
                          for b,e in covering_sessions(s, t)) <= 1
              for d in range(data.n_days)
              for t in range(data.n_times)),
              name='room_double_booking'
             )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

## Other Hard Constraints

In [ ]:
# Every Faculty member can be teaching at most one course at a time
# core courses for the same batch cannot clash

## Soft Constraints

In [ ]:
# preferred time slots for courses / faculty
# at least one day gap between two consecutive sessions of a course
# one constraint for each student --- to ensure there is no clash
# only lab courses have to be slotted at 2PM on Monday or Wednesday (high priority -- though not a hard constraint)

# add the slack variables to the list of variables

In [5]:
def is_lab_tutorial(s:int):
    # Returns true if the session with index 's' is a lab/tutorial course
    return False

lab_tut_slack = tt.addVar(vtype=GRB.BINARY, name="lab_tut_slack")
print(lab_tut_slack.getAttr(GRB.Attr.VarName))

tt.addConstrs((S[b,e,d,s] - lab_tut == 0
              for s in range(n_sessions) if not is_lab_tutorial(s)
              for d, t in data.no_lecture_slots
              for b, e in covering_sessions(s, t)
              ),
              name='no_lecture_slots')

AttributeError: Index out of range for attribute 'VarName'

## Objective Function

In [ ]:
# Minimize Sum
# --- elective clashes
# --- slack for soft constraints (possibly with weights)

In [18]:
weights = {lab_tut_slack.get(gp.): 1}
objective = gp.quicksum(tt.getVarByName(name)*w for name, w in weights.items())
tt.setObjective(objective, sense=GRB.MINIMIZE)

AttributeError: 'gurobipy.Var' object has no attribute 'name'

## Optimize

In [ ]:
tt.optimize()